In [25]:
import pandas as pd

In [26]:
dd_users = pd.read_csv('../../Archivos_tfm/dataTesting/usuarios_DD_1809a1110.csv')
dd_users.head()

,Código de cliente,Usuarios
0,F10001681,1
1,F10004567,1
2,F10007287,1
3,F10019099,1
4,F10020552,1


In [27]:
dd_users.columns

Index([u'Código de cliente', u'Usuarios'], dtype='object')

In [28]:
# Cambiamos el nombre de la columna
dd_users.columns = ['ID','Usuarios']

# Eliminamos la columna usuarios, ya que no la vamos a necesitar
dd_users.drop('Usuarios',1, inplace=True)
dd_users.head()

#Esta lista de usuarios es sobre la que vamos a centrar todo nuestro trabajo e iremos añadiendole columnas.

,ID
0,F10001681
1,F10004567
2,F10007287
3,F10019099
4,F10020552


Ya tenemos los usuarios que han accedido al proceso que nos interesa, limpios y preparados para añadirles contenido útil para el algoritmo.

Tenemos que trabajar de esta manera porque google analytics al intentar sacar todos estos datos ya filtrados por el proceso que nos interesa, digamos que hace cosas raras y perdemos usuarios en el camino. Por lo que tenemos que ir haciendo consultas muy simples, las cuales manipularemos y añadiremos a nuestros queridos usuarios.

# Añadimos la profundidad del proceso
Esta nueva característica nos aporta información acerca de cuan lejos a llegado el usuario en el proceso.

Debemos hacer una consulta en google analytics en la que tengamos:
 - ID usuario
 - Páginas del proceso en las que ha estado el usuario
 - Una métrica neutral (para no perder usuarios) como usuarios.

In [29]:
profundidad = pd.read_csv('../../Archivos_tfm/dataTesting/profundidad.csv', index_col=0)
profundidad.drop('neutral_metric',1,inplace=True)
profundidad.head()
# El csv ya tenía indice porque lo he modificado previamente para no mostrar
# el nombre real de las pantallas de proceso, por eso usamos index_col=0.
# Eliminamos la columna neutral_metric, solo la necesitabamos para hacer la consulta en google analytics.

,ID,page
0,F10001681,page1
1,F10004567,page1
2,F10007287,page2
3,F10007287,page3
4,F10007287,page1


El objetivo es obtener un nuevo dataframe que contenga por columnas... usuarios únicos y profundidad.
Básicamente vamos a hacer el "The word-count problem" con nuestro dataframe. Ya que un usuarios puede aparecer hasta 4 veces... una por cada paso del proceso.

In [30]:
list_users = profundidad['ID']
list_users.head()

0    F10001681
1    F10004567
2    F10007287
3    F10007287
4    F10007287
Name: ID, dtype: object

In [31]:
def countUsers(lista):
    """
    method para contar veces que aparece un usuario
    
    inputs:
        *lista: lista de usuarios
        
    output: diccionario de {'ID usuario':profundidad en el proceso}
    """
    # Creamos un diccionario vacío
    hist = {}
    
    for user in lista:
        if user in hist:
            hist[user] += 1
        else:
            hist[user] = 1
            
    return hist

dict_user = countUsers(list_users)
# Transformamos nuestro diccionario en un dataframe
profundidad = pd.DataFrame(dict_user.items(), columns=['ID','Profundidad'])
profundidad.head()

,ID,Profundidad
0,F7192075,4
1,F5501691,1
2,F9733307,4
3,F13669657,4
4,F10542990,1


Ya tenemos creada la profundidad del proceso, ahora debemos añadir esta nueva caracteristica a dd_users.

In [32]:
dd_users = pd.merge(dd_users,profundidad, on = 'ID')

In [33]:
dd_users

,ID,Profundidad
0,F10001681,1
1,F10004567,1
2,F10007287,4
3,F10019099,4
4,F10020552,1
5,F10024071,1
6,F10026447,4
7,F10027084,1
8,F10029572,4
9,F100318,4
